# SQL Query Engine Llama-Index

### Connecting to MYSQL

In [36]:
from sqlalchemy import create_engine,URL,MetaData,Text

url_object = URL.create(
    "mysql+mysqldb",
    username="root",
    password="<your_password>", 
    host="<your_host>",
    database="classicmodels",
)
engine = create_engine(url_object)

In [ ]:
conn = engine.connection

In [30]:
from sqlalchemy import inspect

insp = inspect(engine)
tables_names=insp.get_table_names()

### Creating SQLDatabase Object

In [26]:
from llama_index.llms.openai import OpenAI

api_key = '' 
%env OPENAI_API_KEY={api_key}

llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo-0125")

env: OPENAI_API_KEY=sk-proj-FGqvJX2CbFZuge9GQBpKT3BlbkFJVSOvDUQglDrc8M8l9ILZ


In [48]:
from llama_index.core.prompts.base import PromptTemplate
from llama_index.core.prompts.prompt_type import PromptType

TEXT_TO_SQL_TMPL = (
    "Given an input question, first create a syntactically correct {dialect} "
    "query to run, then look at the results of the query and return the answer. "
    "You can order the results by a relevant column to return the most "
    "interesting examples in the database.\n\n"
    "Never query for all the columns from a specific table, only ask for a "
    "few relevant columns given the question.\n\n"
    "Pay attention to use only the column names that you can see in the schema "
    "description. "
    "Be careful to not query for columns that do not exist. "
    "Pay attention to which column is in which table. "
    "Also, qualify column names with the table name when needed. "
    "Pay attetntion to use unique Alias for tables."
    "You are required to use the following format, each taking one line:\n\n"
    "Question: Question here\n"
    "SQLQuery: SQL Query to run\n"
    "SQLResult: Result of the SQLQuery\n"
    "Answer: Final answer here\n\n"
    "Only use tables listed below.\n"
    "{schema}\n\n"
    "Question: {query_str}\n"
    "SQLQuery: "
)
TEXT_TO_SQL_PROMPT = PromptTemplate(
    TEXT_TO_SQL_TMPL,
    prompt_type=PromptType.TEXT_TO_SQL,
)

In [49]:
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex
from llama_index.core import SQLDatabase
metadata_obj = MetaData()
metadata_obj.create_all(engine)
sql_database = SQLDatabase(engine, include_tables=tables_names)

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [(SQLTableSchema(table_name=table_name_)) for table_name_ in tables_names]# add a SQLTableSchema for each table
obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=3),llm=llm,text_to_sql_prompt=TEXT_TO_SQL_PROMPT
)

## Questions

In [ ]:
import time
import pandas as pd

df_benchmark = pd.DataFrame(columns=['question','answer','sql','time'])
n_tests = 1

### 1. schema

In [32]:
query_str="List the top 5 sales representatives in each office based on their cumulative sales."

add_times=0
n_test = n_tests
for _ in range(n_test):
    start = time.time()
    response = query_engine.query(query_str)
    end = time.time()
    add_times+=end-start

print('time: ',add_times/n_test)
print('Model Answer: \n')
print(response.response)
print('\nSQL: \n')
print(response.metadata['sql_query'])

row = [query_str, response, response.metadata['sql_query'],add_times/n_test]
df_benchmark.loc[len(df_benchmark)] = row


time:  12.599762678146362
Model Answer: 

The top 5 sales representatives in each office based on their cumulative sales are as follows:
- In the San Francisco office, Leslie Jennings has total sales of $989,906.55 and Leslie Thompson has total sales of $347,533.03.
- In the Boston office, Steve Patterson has total sales of $449,219.13 and Julie Firrelli has total sales of $386,663.20.
- In the NYC office, George Vanauf has total sales of $584,406.80.

SQL: 

SELECT e.employeeNumber, e.firstName, e.lastName, o.officeCode, o.city, SUM(p.amount) AS total_sales
FROM employees e
JOIN offices o ON e.officeCode = o.officeCode
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN payments p ON c.customerNumber = p.customerNumber
GROUP BY e.employeeNumber, o.officeCode
ORDER BY o.officeCode, total_sales DESC
LIMIT 5;


### 2. SELECT

In [64]:
query_str = "How many clientes are from Dinamarca" 

add_times=0
n_test = n_tests
for _ in range(n_test):
    start = time.time()
    response = query_engine.query(query_str)
    end = time.time()
    add_times+=end-start

print('time: ',add_times/n_test)
print('Model Answer: \n')
print(response.response)
print('\nSQL: \n')
print(response.metadata['sql_query'])

row = [query_str, response, response.metadata['sql_query'],add_times/n_test]
df_benchmark.loc[len(df_benchmark)] = row


time:  7.1656646728515625
Model Answer: 

There are 2 customers from Denmark.

SQL: 

SELECT COUNT(customerNumber) AS num_customers
FROM customers
WHERE country = 'Denmark';


### 3. JOIN table

In [19]:
query_str = "List the 10 lasts orders pair with the customer name and order date." 

add_times=0
n_test = n_tests

for _ in range(n_test):
    start = time.time()
    response = query_engine.query(query_str)
    end = time.time()
    add_times+=end-start

print('time: ',add_times/n_test)
print('Model Answer: \n')
print(response.response)
print('\nSQL: \n')
print(response.metadata['sql_query'])

row = [query_str, response, response.metadata['sql_query'],add_times/n_test]
df_benchmark.loc[len(df_benchmark)] = row

time:  14.020272731781006
Model Answer: 

The 10 latest orders are as follows:
1. Order Number: 10424, Customer Name: Euro+ Shopping Channel, Order Date: May 31, 2005
2. Order Number: 10425, Customer Name: La Rochelle Gifts, Order Date: May 31, 2005
3. Order Number: 10422, Customer Name: Diecast Classics Inc., Order Date: May 30, 2005
4. Order Number: 10423, Customer Name: Petit Auto, Order Date: May 30, 2005
5. Order Number: 10420, Customer Name: Souveniers And Things Co., Order Date: May 29, 2005
6. Order Number: 10421, Customer Name: Mini Gifts Distributors Ltd., Order Date: May 29, 2005
7. Order Number: 10419, Customer Name: Salzburg Collectables, Order Date: May 17, 2005
8. Order Number: 10418, Customer Name: Extreme Desk Decorations, Ltd, Order Date: May 16, 2005
9. Order Number: 10417, Customer Name: Euro+ Shopping Channel, Order Date: May 13, 2005
10. Order Number: 10416, Customer Name: L'ordine Souveniers, Order Date: May 10, 2005

SQL: 

SELECT o.orderNumber, c.customerName, 

### 4. JOIN and SUM

In [20]:
query_str = "Find the total amount spent by the top 10 customer." 
add_times=0
n_test = n_tests

for _ in range(n_test):
    start = time.time()
    response = query_engine.query(query_str)
    end = time.time()
    add_times+=end-start

print('time: ',add_times/n_test)
print('Model Answer: \n')
print(response.response)
print('\nSQL: \n')
print(response.metadata['sql_query'])

row = [query_str, response, response.metadata['sql_query'],add_times/n_test]
df_benchmark.loc[len(df_benchmark)] = row

time:  15.206795454025269
Model Answer: 

The total amount spent by the top 10 customers are as follows:
1. Euro+ Shopping Channel: $715,738.98
2. Mini Gifts Distributors Ltd.: $584,188.24
3. Australian Collectors, Co.: $180,585.07
4. Muscle Machine Inc: $177,913.95
5. Dragon Souveniers, Ltd.: $156,251.03
6. Down Under Souveniers, Inc: $154,622.08
7. AV Stores, Co.: $148,410.09
8. Anna's Decorations, Ltd: $137,034.22
9. Corporate Gift Ideas Co.: $132,340.78
10. Saveley & Henriot, Co.: $130,305.35

SQL: 

SELECT c.customerNumber, c.customerName, SUM(p.amount) AS total_amount
FROM customers c
JOIN payments p ON c.customerNumber = p.customerNumber
GROUP BY c.customerNumber, c.customerName
ORDER BY total_amount DESC
LIMIT 10;


### 5.

In [21]:
query_str = "Show the total quantity of each product ordered by customers along with their corresponding product codes and names." 
add_times=0
n_test = n_tests

for _ in range(n_test):
    start = time.time()
    response = query_engine.query(query_str)
    end = time.time()
    add_times+=end-start

print('time: ',add_times/n_test)
print('Model Answer: \n')
print(response.response)
print('\nSQL: \n')
print(response.metadata['sql_query'])

row = [query_str, response, response.metadata['sql_query'],add_times/n_test]
df_benchmark.loc[len(df_benchmark)] = row

time:  10.256795644760132
Model Answer: 

The SQL query provided is invalid. Please review the syntax and structure of the query to ensure it is correctly formatted.

SQL: 

SELECT productCode, productName, SUM(quantityOrdered) AS totalQuantity
FROM products
JOIN orderdetails ON products.productCode = orderdetails.productCode
GROUP BY productCode, productName
ORDER BY totalQuantity DESC;


### 6.

In [22]:
query_str = "How does the average payment amount vary based on the customer's credit limit?" 
add_times=0
n_test = n_tests

for _ in range(n_test):
    start = time.time()
    response = query_engine.query(query_str)
    end = time.time()
    add_times+=end-start

print('time: ',add_times/n_test)
print('Model Answer: \n')
print(response.response)
print('\nSQL: \n')
print(response.metadata['sql_query'])

row = [query_str, response, response.metadata['sql_query'],add_times/n_test]
df_benchmark.loc[len(df_benchmark)] = row

time:  10.90698516368866
Model Answer: 

It appears that there was an error in the SQL query provided. Please double-check the syntax and try running the query again to obtain the average payment amount based on the customer's credit limit.

SQL: 

SELECT c.customerNumber, c.creditLimit, AVG(p.amount) AS avg_payment_amount
FROM customers c
JOIN payments p ON c.customerNumber = p.customerNumber
GROUP BY c.creditLimit
ORDER BY avg_payment_amount DESC;


### 7. using two joins

In [23]:
query_str = "Retrieve the names of employees and their managers from San Francisco" 
add_times=0
n_test = n_tests

for _ in range(n_test):
    start = time.time()
    response = query_engine.query(query_str)
    end = time.time()
    add_times+=end-start

print('time: ',add_times/n_test)
print('Model Answer: \n')
print(response.response)
print('\nSQL: \n')
print(response.metadata['sql_query'])

row = [query_str, response, response.metadata['sql_query'],add_times/n_test]
df_benchmark.loc[len(df_benchmark)] = row


time:  36.17472290992737
Model Answer: 

The employees and their managers from San Francisco are as follows:
- Anthony Bow is managed by Mary Patterson
- Jeff Firrelli is managed by Diane Murphy
- Leslie Jennings is managed by Anthony Bow
- Mary Patterson is managed by Diane Murphy
- Leslie Thompson is managed by Anthony Bow

SQL: 

SELECT e1.firstName AS employeeFirstName, e1.lastName AS employeeLastName, e2.firstName AS managerFirstName, e2.lastName AS managerLastName
FROM employees e1
JOIN employees e2 ON e1.reportsTo = e2.employeeNumber
JOIN offices o ON e1.officeCode = o.officeCode
WHERE o.city = 'San Francisco'
ORDER BY e1.lastName;


### 8.

In [24]:
query_str = "List the employee names and their corresponding office cities for employees who have customers located in the USA." 
add_times=0
n_test = n_tests

for _ in range(n_test):
    start = time.time()
    response = query_engine.query(query_str)
    end = time.time()
    add_times+=end-start

print('time: ',add_times/n_test)
print('Model Answer: \n')
print(response.response)
print('\nSQL: \n')
print(response.metadata['sql_query'])

row = [query_str, response, response.metadata['sql_query'],add_times/n_test]
df_benchmark.loc[len(df_benchmark)] = row

time:  38.10796892642975
Model Answer: 

Employees with customers located in the USA and their corresponding office cities are:
- Julie Firrelli in Boston
- Leslie Jennings in San Francisco
- Steve Patterson in Boston
- Leslie Thompson in San Francisco
- Foon Yue Tseng in NYC
- George Vanauf in NYC

SQL: 

SELECT e.firstName, e.lastName, o.city
FROM employees e
JOIN offices o ON e.officeCode = o.officeCode
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
WHERE c.country = 'USA'
ORDER BY e.lastName, e.firstName;


### 9. count, having count

In [25]:
query_str = "List the product lines with the total number of products in each line, showing only those product lines having more than 5 products." 
add_times=0
n_test = n_tests

for _ in range(n_test):
    start = time.time()
    response = query_engine.query(query_str)
    end = time.time()
    add_times+=end-start

print('time: ',add_times/n_test)
print('Model Answer: \n')
print(response.response)
print('\nSQL: \n')
print(response.metadata['sql_query'])

row = [query_str, response, response.metadata['sql_query'],add_times/n_test]
df_benchmark.loc[len(df_benchmark)] = row


time:  8.576993107795715
Model Answer: 

The product lines with more than 5 products are Classic Cars with 38 products, Vintage Cars with 24 products, Motorcycles with 13 products, Planes with 12 products, Trucks and Buses with 11 products, and Ships with 9 products.

SQL: 

SELECT productLine, COUNT(productCode) AS totalProducts
FROM products
GROUP BY productLine
HAVING COUNT(productCode) > 5
ORDER BY totalProducts DESC;


### 10. long output, missing full data in the output

In [26]:
query_str = "Display product details along with their product line and supplier information." 
add_times=0
n_test = n_tests

for _ in range(n_test):
    start = time.time()
    response = query_engine.query(query_str)
    end = time.time()
    add_times+=end-start

print('time: ',add_times/n_test)
print('Model Answer: \n')
print(response.response)
print('\nSQL: \n')
print(response.metadata['sql_query'])

row = [query_str, response, response.metadata['sql_query'],add_times/n_test]
df_benchmark.loc[len(df_benchmark)] = row


time:  115.34193754196167
Model Answer: 

Here are some product details along with their product line and supplier information:

1. Product Name: 1952 Citroen-15CV
   - Product Description: Precision crafted hand-assembled 1:18 scale reproduction of the 1952 15CV, with detailed features.
   - Product Line: Classic Cars
   - Supplier: Exoto Designs

2. Product Name: 1948 Porsche Type 356 Roadster
   - Product Description: Features working front and rear suspension, opening engine cover, and more.
   - Product Line: Classic Cars
   - Supplier: Gearbox Collectibles

3. Product Name: 1970 Triumph Spitfire
   - Product Description: Features opening and closing doors in white color.
   - Product Line: Classic Cars
   - Supplier: Min Lin Diecast

These are just a few examples of the product details, product line, and supplier information available in the database.

SQL: 

SELECT p.productName, p.productDescription, p.productLine, p.productVendor, pl.textDescription
FROM products p
JOIN produc

### 11. three tables, count and avg

In [27]:
query_str = "How many orders has each customer placed, and what is the average order value?" 
add_times=0
n_test = n_tests

for _ in range(n_test):
    start = time.time()
    response = query_engine.query(query_str)
    end = time.time()
    add_times+=end-start

print('time: ',add_times/n_test)
print('Model Answer: \n')
print(response.response)
print('\nSQL: \n')
print(response.metadata['sql_query'])

row = [query_str, response, response.metadata['sql_query'],add_times/n_test]
df_benchmark.loc[len(df_benchmark)] = row

time:  12.035310983657837
Model Answer: 

The customer with the highest number of orders is Euro+ Shopping Channel with 259 orders and an average order value of $3168.69. Mini Gifts Distributors Ltd. follows closely with 180 orders and an average order value of $3287.93. Australian Collectors, Co. has placed 55 orders with an average order value of $3283.36. These are the top three customers in terms of order frequency and average order value.

SQL: 

SELECT c.customerNumber, c.customerName, COUNT(o.orderNumber) AS numOrders, AVG(od.quantityOrdered * od.priceEach) AS avgOrderValue
FROM customers c
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY c.customerNumber, c.customerName
ORDER BY numOrders DESC;


### 12. four tables

In [28]:
query_str = "Show the customer name, order date, product name, and quantity ordered for the last 10 orders." 
add_times=0
n_test = n_tests

for _ in range(n_test):
    start = time.time()
    response = query_engine.query(query_str)
    end = time.time()
    add_times+=end-start

print('time: ',add_times/n_test)
print('Model Answer: \n')
print(response.response)
print('\nSQL: \n')
print(response.metadata['sql_query'])

row = [query_str, response, response.metadata['sql_query'],add_times/n_test]
df_benchmark.loc[len(df_benchmark)] = row

time:  13.125012278556824
Model Answer: 

The last 10 orders included products such as the 1952 Alpine Renault 1300, 1958 Setra Bus, 1940 Ford Pickup Truck, 1939 Cadillac Limousine, 1996 Peterbilt 379 Stake Bed with Outrigger, 1982 Camaro Z28, 1962 LanciaA Delta 16V, 1957 Chevy Pickup, 1998 Chrysler Plymouth Prowler, and 1964 Mercedes Tour Bus. These orders were made by customers from Euro+ Shopping Channel and La Rochelle Gifts on May 31, 2005. The quantities ordered varied between 26 and 54 units per product.

SQL: 

SELECT c.customerName, o.orderDate, p.productName, od.quantityOrdered
FROM customers c
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
JOIN products p ON od.productCode = p.productCode
ORDER BY o.orderDate DESC
LIMIT 10;


### 13. four tables

In [29]:
query_str = "Retrieve the product names and descriptions for the 5 products that have been ordered the most by customers located in the USA." 
add_times=0
n_test = n_tests

for _ in range(n_test):
    start = time.time()
    response = query_engine.query(query_str)
    end = time.time()
    add_times+=end-start

print('time: ',add_times/n_test)
print('Model Answer: \n')
print(response.response)
print('\nSQL: \n')
print(response.metadata['sql_query'])

row = [query_str, response, response.metadata['sql_query'],add_times/n_test]
df_benchmark.loc[len(df_benchmark)] = row



time:  11.870614171028137
Model Answer: 

The top 5 products ordered by customers in the USA are the America West Airlines B757-200, F/A 18 Hornet 1/72, 1917 Grand Touring Sedan, 1949 Jaguar XK 120, and 1957 Ford Thunderbird. These products range from model airplanes to classic car replicas, each with intricate details and features that make them popular among customers.

SQL: 

SELECT p.productName, p.productDescription
FROM products p
JOIN orderdetails od ON p.productCode = od.productCode
JOIN orders o ON od.orderNumber = o.orderNumber
WHERE o.customerNumber IN (SELECT customerNumber FROM customers WHERE country = 'USA')
ORDER BY od.quantityOrdered DESC
LIMIT 5;


### 14

In [30]:
query_str = "Calculate the average number of days it takes to ship each product from the order date to the shipped date."
add_times=0
n_test = n_tests

for _ in range(n_test):
    start = time.time()
    response = query_engine.query(query_str)
    end = time.time()
    add_times+=end-start

print('time: ',add_times/n_test)
print('Model Answer: \n')
print(response.response)
print('\nSQL: \n')
print(response.metadata['sql_query'])

#fixed question= "Calculate the average number of days it takes to ship each product"

time:  11.101412773132324
Model Answer: 

The average number of days it takes to ship each product from the order date to the shipped date ranges from approximately 3 to 5.8 days. Products like the 1970 Triumph Spitfire and the 2001 Ferrari Enzo have longer average shipping times, while items like the 1940 Ford Delivery Sedan have shorter average shipping times.

SQL: 

SELECT p.productCode, p.productName, AVG(DATEDIFF(o.shippedDate, o.orderDate)) AS avg_shipping_days
FROM products p
JOIN orderdetails od ON p.productCode = od.productCode
JOIN orders o ON od.orderNumber = o.orderNumber
WHERE o.shippedDate IS NOT NULL
GROUP BY p.productCode, p.productName
ORDER BY avg_shipping_days DESC;


### 15.

In [31]:
query_str = "Find the number of customers who have placed more than one order in USA" 

response = query_engine.query(query_str)
add_times=0
n_test = n_tests

for _ in range(n_test):
    start = time.time()
    response = query_engine.query(query_str)
    end = time.time()
    add_times+=end-start

print('time: ',add_times/n_test)
print('Model Answer: \n')
print(response.response)
print('\nSQL: \n')
print(response.metadata['sql_query'])

row = [query_str, response, response.metadata['sql_query'],add_times/n_test]
df_benchmark.loc[len(df_benchmark)] = row

time:  8.822625279426575
Model Answer: 

There are 34 customers who have placed more than one order in the USA.

SQL: 

SELECT COUNT(DISTINCT c.customerNumber) AS num_customers
FROM customers c
JOIN orders o ON c.customerNumber = o.customerNumber
WHERE c.country = 'USA'
GROUP BY c.customerNumber
HAVING COUNT(o.orderNumber) > 1


### 16

In [32]:
import time
query_str = "Show me the total amount sold comparison between the product line Cars vs Trains"

response = query_engine.query(query_str)
add_times=0
n_test = n_tests

for _ in range(n_test):
    start = time.time()
    response = query_engine.query(query_str)
    end = time.time()
    add_times+=end-start

print('time: ',add_times/n_test)
print('Model Answer: \n')
print(response.response)
print('\nSQL: \n')
print(response.metadata['sql_query'])

row = [query_str, response, response.metadata['sql_query'],add_times/n_test]
df_benchmark.loc[len(df_benchmark)] = row

time:  8.941961407661438
Model Answer: 

The total amount sold for the product line Trains is 2818. Unfortunately, there is no data available for the total amount sold for the product line Cars in the query results.

SQL: 

SELECT p.productLine, SUM(od.quantityOrdered) AS totalAmountSold
FROM products p
JOIN orderdetails od ON p.productCode = od.productCode
WHERE p.productLine IN ('Cars', 'Trains')
GROUP BY p.productLine
ORDER BY totalAmountSold DESC;


### 17.

In [33]:
query_str = "Who are the Sales Representative from US?"

response = query_engine.query(query_str)
add_times=0
n_test = n_tests

for _ in range(n_test):
    start = time.time()
    response = query_engine.query(query_str)
    end = time.time()
    add_times+=end-start

print('time: ',add_times/n_test)
print('Model Answer: \n')
print(response.response)
print('\nSQL: \n')
print(response.metadata['sql_query'])

row = [query_str, response, response.metadata['sql_query'],add_times/n_test]
df_benchmark.loc[len(df_benchmark)] = row

time:  29.070027709007263
Model Answer: 

The Sales Representatives from the US are Julie Firrelli, Leslie Jennings, Steve Patterson, Leslie Thompson, Foon Yue Tseng, and George Vanauf.

SQL: 

SELECT e.employeeNumber, e.lastName, e.firstName, e.jobTitle, e.email
FROM employees e
JOIN offices o ON e.officeCode = o.officeCode
WHERE e.jobTitle = 'Sales Rep' AND o.country = 'USA'
ORDER BY e.lastName, e.firstName;


In [34]:
df_benchmark.to_csv('current-2.csv')